In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Save your Streamlit app code to a file
streamlit_script = """

import streamlit as st
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the trained model (Update path accordingly)
model_path = '/content/drive/MyDrive/Dataset/Mall_Dataset/models/time_stamp_model.keras'  # Path to your saved model
model = load_model(model_path)  # Load the model

# Function to load and preprocess the image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path)
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict the total people count and zone-wise people count
def run_detector(model, image_path):
    # Load and preprocess the image
    img_array = load_and_preprocess_image(image_path)

    # Predict the total people count (assuming output is a vector of counts for total and zones)
    predictions = model.predict(img_array)

    # Extract the predictions for total count and zones (5 values: people_count, zone_1, zone_2, zone_3, zone_4)
    people_count_pred, zone_1_pred, zone_2_pred, zone_3_pred, zone_4_pred = predictions[0]

    # Round the predictions to the nearest integer
    people_count_pred = round(people_count_pred)
    zone_1_pred = round(zone_1_pred)
    zone_2_pred = round(zone_2_pred)
    zone_3_pred = round(zone_3_pred)
    zone_4_pred = round(zone_4_pred)

    # Return the predictions
    return people_count_pred, zone_1_pred, zone_2_pred, zone_3_pred, zone_4_pred

# Function to determine the business level based on total people count
def get_business_level(total_people_count):
    if total_people_count < 10:
        return "Low"
    elif 10 <= total_people_count < 20:
        return "Moderate"
    else:
        return "High"

# Streamlit UI for image upload and model interaction
st.title("Improve Mall Traffic with AI-Powered People Counting")

# File uploader widget
uploaded_image = st.file_uploader("Upload Image", type=["jpg", "png", "jpeg"])

if uploaded_image is not None:
    # Save the uploaded image to a temporary file
    image_path = "/content/tmp_uploaded_image.jpg"
    with open(image_path, "wb") as f:
        f.write(uploaded_image.getbuffer())

    # Run the detector and get predictions
    people_count_pred, zone_1_pred, zone_2_pred, zone_3_pred, zone_4_pred = run_detector(model, image_path)

    # Display the total people count
    st.write(f"**Total People Detected:** {people_count_pred}")

    # Display zone-wise people count
    st.sidebar.subheader("Zone-wise People Count")
    st.sidebar.write(f"**Zone 1:** {zone_1_pred} people")
    st.sidebar.write(f"**Zone 2:** {zone_2_pred} people")
    st.sidebar.write(f"**Zone 3:** {zone_3_pred} people")
    st.sidebar.write(f"**Zone 4:** {zone_4_pred} people")

    # Display the business level of the mall
    business_level = get_business_level(people_count_pred)
    st.write(f"**Mall Business Level:** {business_level}")

    # Display the uploaded image
    image_pil = Image.open(image_path)
    st.image(image_pil, caption="Uploaded Image", use_container_width=True)

    # Display the result (image with zones and counts)
    img_display = cv2.imread(image_path)
    img_display = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct display

    # Zone height and width based on 2x2 grid
    zone_height = img_display.shape[0] // 2
    zone_width = img_display.shape[1] // 2

    # Draw rectangles for each zone and label it with the zone number
    for zone_index in range(1, 5):  # For 4 zones
        row = (zone_index - 1) // 2  # Determine the row for the zone (0 or 1)
        col = (zone_index - 1) % 2   # Determine the column for the zone (0 or 1)

        # Calculate the coordinates of the top-left and bottom-right corners of the zone
        y1 = row * zone_height
        y2 = (row + 1) * zone_height
        x1 = col * zone_width
        x2 = (col + 1) * zone_width

        # Draw the rectangle on the image
        cv2.rectangle(img_display, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Add the zone number text inside the rectangle
        cv2.putText(img_display, f'Zone {zone_index}', (x1 + 10, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display the processed image with the bounding boxes and zone numbers
    st.image(img_display, caption=f"Processed Image with {people_count_pred} People Detected", use_container_width=True)

"""

with open("streamlit_app.py", "w") as f:
    f.write(streamlit_script)

In [9]:
!pip install streamlit pyngrok tensorflow pillow --quiet

In [10]:
from pyngrok import ngrok

# Set your ngrok authentication token
ngrok.set_auth_token("2pGVlynRP3dsCGA3U8VPqImSktv_6VZj5BppdZe9ZDEuhiX2N")

# Start the ngrok tunnel
public_url = ngrok.connect(8501, "http")
print(f"Public URL for Streamlit: {public_url}")


# Run the Streamlit app in the background
!streamlit run streamlit_app.py &

In [ ]:
!jupyter nbconvert --to html /content/DATA_CLEANING_AND_EDA_TASK_1.ipynb